# --- Day 19: Aplenty ---
The Elves of Gear Island are thankful for your help and send you on your way. They even have a hang glider that someone stole from Desert Island; since you're already going that direction, it would help them a lot if you would use it to get down there and return it to them.

As you reach the bottom of the relentless avalanche of machine parts, you discover that they're already forming a formidable heap. Don't worry, though - a group of Elves is already here organizing the parts, and they have a system.

To start, each part is rated in each of four categories:
```
x: Extremely cool looking
m: Musical (it makes a noise when you hit it)
a: Aerodynamic
s: Shiny
```
Then, each part is sent through a series of workflows that will ultimately accept or reject the part. Each workflow has a name and contains a list of rules; each rule specifies a condition and where to send the part if the condition is true. The first rule that matches the part being considered is applied immediately, and the part moves on to the destination described by the rule. (The last rule in each workflow has no condition and always applies if reached.)

Consider the workflow ex{x>10:one,m<20:two,a>30:R,A}. This workflow is named ex and contains four rules. If workflow ex were considering a specific part, it would perform the following steps in order:

Rule "x>10:one": If the part's x is more than 10, send the part to the workflow named one.
Rule "m<20:two": Otherwise, if the part's m is less than 20, send the part to the workflow named two.
Rule "a>30:R": Otherwise, if the part's a is more than 30, the part is immediately rejected (R).
Rule "A": Otherwise, because no other rules matched the part, the part is immediately accepted (A).
If a part is sent to another workflow, it immediately switches to the start of that workflow instead and never returns. If a part is accepted (sent to A) or rejected (sent to R), the part immediately stops any further processing.

The system works, but it's not keeping up with the torrent of weird metal shapes. The Elves ask if you can help sort a few parts and give you the list of workflows and some part ratings (your puzzle input). For example:
```
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
```
The workflows are listed first, followed by a blank line, then the ratings of the parts the Elves would like you to sort. All parts begin in the workflow named in. In this example, the five listed parts go through the following workflows:
```
{x=787,m=2655,a=1222,s=2876}: in -> qqz -> qs -> lnx -> A
{x=1679,m=44,a=2067,s=496}: in -> px -> rfg -> gd -> R
{x=2036,m=264,a=79,s=2244}: in -> qqz -> hdj -> pv -> A
{x=2461,m=1339,a=466,s=291}: in -> px -> qkq -> crn -> R
{x=2127,m=1623,a=2188,s=1013}: in -> px -> rfg -> A
```
Ultimately, three parts are accepted. Adding up the x, m, a, and s rating for each of the accepted parts gives 7540 for the part with x=787, 4623 for the part with x=2036, and 6951 for the part with x=2127. Adding all of the ratings for all of the accepted parts gives the sum total of 19114.

Sort through all of the parts you've been given; what do you get if you add together all of the rating numbers for all of the parts that ultimately get accepted?

## --- Part Two ---
Even with your help, the sorting process still isn't fast enough.

One of the Elves comes up with a new plan: rather than sort parts individually through all of these workflows, maybe you can figure out in advance which combinations of ratings will be accepted or rejected.

Each of the four ratings (x, m, a, s) can have an integer value ranging from a minimum of 1 to a maximum of 4000. Of all possible distinct combinations of ratings, your job is to figure out which ones will be accepted.

In the above example, there are 167409079868000 distinct combinations of ratings that will be accepted.

Consider only your list of workflows; the list of part ratings that the Elves wanted you to sort is no longer relevant. How many distinct combinations of ratings will be accepted by the Elves' workflows?

In [1]:
from pathlib import Path
import os

yr = 2023
d = 19

inp_path = os.path.join(Path(os.path.abspath("")).parents[1], 
             'Input', '{}'.format(yr), 
             '{}.txt'.format(d))


with open(inp_path, 'r') as file:
    inp = file.read()

In [2]:
def format_input(inp):
  formatted_input = {'workflows': {}, 'parts': []}
  done_with_workflows = False
  for l in inp.splitlines():
    if len(l) == 0:
      done_with_workflows = True
      continue
    if not done_with_workflows:
      workflow_name = l.split('{')[0]
      workflow_info = l.split('{')[1].replace('}','')
      workflow_instructions = workflow_info.split(',')
      cur_workflow_tests = []
      for inst in workflow_instructions:
        if '>' in inst:
          comp = '>'
          c = inst.split('>')[0]
          val = int(inst.split('>')[1].split(':')[0])
          dest = inst.split('>')[1].split(':')[1]
          workflow_func = lambda x, c=c, val=val: x[c]>val
        elif '<' in inst:
          comp = '<'
          c = inst.split('<')[0]
          val = int(inst.split('<')[1].split(':')[0])
          dest = inst.split('<')[1].split(':')[1]
          workflow_func = lambda x, c=c, val=val: x[c]<val
        else:
          c = None
          comp = None
          val = None
          dest = inst
          workflow_func = lambda x: True
        cur_workflow_tests.append({'test': workflow_func,
                                   'dest': dest,
                                   'c': c,
                                   'comp': comp,
                                   'val': val})
      formatted_input['workflows'][workflow_name] = cur_workflow_tests
    else:
      part = {s.split('=')[0]: int(s.split('=')[1])
              for s in l.replace('}','').replace('{','').split(',')}
      formatted_input['parts'].append(part)
  return formatted_input

In [3]:
def process_part(workflows, part, debug=False):
  '''
  Workflows are saved as lists of dictionaries
  containing lambda functions that perform the test
  as well as destination values which denote
  where the part will be sent to upon a positive
  test result (this will be either 'A', 'R', or
  another workflow). Starting at the 'in'
  workflow, send the part through the workflows until
  we arrive at 'A', or 'R'
  '''
  def apply_workflow(workflow, part):
    for w in workflow:
      if w['test'](part):
        return w['dest']

  res = apply_workflow(workflows['in'], part)
  while res not in ['A', 'R']:
    res = apply_workflow(workflows[res], part)
  return res=='A'

In [4]:
def sum_of_accepted(formatted_input):
  '''
  Just process each part and add up their category values
  '''
  s = 0
  for p in formatted_input['parts']:
    if process_part(formatted_input['workflows'], p):
      s += sum(p.values())
  return s

In [5]:
def find_distinct_combos(formatted_input):
  '''
  Start with the full possible ranges (1-4000 for all 4 categories)
  Work through the workflows recursively:
    To process one workflow loop over the tests within each workflow
    in order keeping a rolling sum of potential combos.
    At each test IF the test sends you to 'A', add the total number of combos
    yielded by the current possible ranges that trigger that test. IF the test
    sends you to 'R', add 0 to the sum since those are not accepted combos, IF
    the current test sends you to another workflow, then add the result of
    recursing on that workflow to our rolling sum with only the ranges from our
    current range that trigger that workflow.
    At the end of each iteration of the test loop, set the current ranges that
    we are examining to only include those ranges that are NOT triggered by
    the previous test.
  '''
  import numpy as np
  from copy import deepcopy

  def trim_range(test, ranges, negate=False):
    '''
    Given a dictionary of ranges trim those ranges to either contain only the
    ranges that trigger the current test, or (if negate==True) only
    the ranges that do NOT trigger the current test
    '''
    ranges = deepcopy(ranges)

    r = test['c']
    if r is None:
      return ranges

    val = test['val']
    comp = test['comp']
    cur_r = ranges[r]
    if comp == '>':
      cur_r = (val+1, cur_r[1]) if not negate else (cur_r[0], val)
    elif comp == '<':
      cur_r = (cur_r[0], val-1) if not negate else (val, cur_r[1])
    else:
      raise Exception('Trying to Trim with an Invalid Comparison')
    ranges[r] = cur_r
    return ranges

  def count_of_ranges(ranges):
    '''
    Get the total number of combos yielded by a specific set of ranges
    '''
    return np.prod(np.array([(ranges[r][1]-ranges[r][0]+1) for r in 'xmas']), dtype=np.ulonglong)


  def _find_distinct_combos(workflows, cur_workflow, cur_ranges):
    '''
    This is the function that we will recurse over
    '''

    # Not sure if we need this but just to be safe
    if any([(cur_ranges[r][1]-cur_ranges[r][0])<0 for r in 'xmas']):
      return 0

    # Just doing this to avoid overflows on our sum
    s = np.array([0], dtype=np.ulonglong)
    for t in workflows[cur_workflow]:
      if t['dest'] == 'A':
        # Add the number of combos in the given ranges and trim our ranges
        s[0] += count_of_ranges(trim_range(t, cur_ranges))
      elif t['dest'] == 'R':
        # These aren't valid combos so just pass
        s[0] += 0
      else:
        # Add the result of the recursion
        s[0] += _find_distinct_combos(workflows,
                                   cur_workflow=t['dest'],
                                   cur_ranges=trim_range(t, cur_ranges))

      # If we made it to the current test in the workflow
      # then we need to adjust our ranges to exclude combos that
      # got caught by the previous test
      cur_ranges = trim_range(t, cur_ranges, negate=True)
    return s[0]


  workflows = formatted_input['workflows']

  return _find_distinct_combos(workflows,
                               cur_workflow = 'in',
                               cur_ranges = {'x':(1,4000),
                                            'm':(1,4000),
                                            'a':(1,4000),
                                            's':(1,4000)}
                              )

In [6]:
import time

t = time.time()

formatted_input = format_input(inp)

print(sum_of_accepted(formatted_input))
print(find_distinct_combos(formatted_input))


print('\nRUNTIME: ', time.time()-t)

263678
125455345557345

RUNTIME:  0.10571646690368652
